# Лекция 04. Алгоритмы, основанные на полезностях. Алгоритм SARSA

State — Action — Reward — State — Action

## Введение

<img src="pictures/niranjan.jpg" alt="Mahesan Niranjan" width="300">

Махесан Ниранджан (Mahesan Niranjan)

- Rummery G. A., Niranjan M. On-line $Q$-learning using connectionistsystems. Technical Report. University of Cambrige. 1994. URL: https://www.researchgate.net/publication/2500611_On-Line_Q-Learning_Using_Connectionist_Systems

<img src="pictures/rummery.jpg" alt="Rummery G. A." width="300">

Руммери Г. А. (Rummery G. A.) ???

https://www.researchgate.net/scientific-contributions/G-A-Rummery-6612457

<img src="pictures/sutton.jpg" alt="Richard Satton" width="300">

Ричард Саттон (Richard S. Sutton)

## $Q$-функция и $V$-функция

Рассмотрим

$$
Q^{\pi}\left(s,a\right)=\mathbb{E}_{s_0=s,\, a_0=a,\, \tau\sim\pi}\left[\sum_{t=0}^{T}\gamma^{t}r_t\right],
$$

$$
V^{\pi}\left(s\right)=\mathbb{E}_{s_0=s,\,\tau\sim\pi}\left[\sum_{t=0}^{T}\gamma^{t}r_t\right].
$$

Функция $Q^{\pi}\left(s,a\right)$ связана с $V^{\pi}\left(s\right)$:

$$
V^{\pi}\left(s\right)=\mathbb{E}_{a\sim\pi\left(s\right)}\left[Q^{\pi}(s,a)\right].
$$

Встает вопрос, какую же функцию лучше настраивать агенту??? $Q^{\pi}\left(s,a\right)$ или $V^{\pi}(s)$???

**Пример.** Предположим, что выбор действия $a$ в соостоянии $s$ приводит к:
- состоянию $s'_1$ с вознаграждением $r_1'=1$,
- состоянию $s'_2$ с вознаграждением $r_2'=2$.

Также известны полезности для $s'_1$ и $s'_2$, $V^{\pi}\left(s_1'\right)=10$ и $V^{\pi}\left(s_2'\right)=-10$, тогда
$$
r_1'+V^{\pi}\left(s_1'\right)=1+10=11,
$$
$$
r_2'+V^{\pi}\left(s_2'\right)=2-10=-8.
$$

Тогда ожидаемая полезность
$$
\mathbb{E}\left[r+V^{\pi}\left(s'\right)\right]=\frac{1}{2}\left(r_1'+V^{\pi}\left(s'_1\right)+r_2'+V^{\pi}\left(s'_2\right)\right)=\frac{1}{2}\left(11-8\right)=1.5.
$$

Однако если пример один, то оценка $\mathbb{E}\left[r+V^{\pi}\left(s'\right)\right]$ будет равна $11$ или $-8$, что далеко от ожидаемого значения $1.5$.

## Метод временных различий (TD-обучение)

В SARSA целевые значения $Q_{\text{tar}}$ порождаются с помощью TD-обучения.

Основная суть использования метода — использование нейронной сети (**сети полезностей**, **value network**), которая на входе оценивает $Q$-значения заданных пар $(s,a)$.

- Генерируются $\tau$ и даются прогнозные $\hat Q$-значения для $\forall (s,a)$.
- С помощью $\tau$ генерируются целевые $Q$-значения $Q_{\text{tar}}$.
- Минимизируются расстояния между $\hat Q$ и $Q_{\text{tar}}$ с помощью стандартной регрессионной функции потреь, такой как скп.
- Процесс повторяем многократно.

### Почему не метод Монте-Карло?

Пусть даны $N$ траекторий $\tau_i, i=1,2,\ldots,N$, начиная с состояния $s$, в котором агент выбрал дейтвие $a$. Тогда оценка по методу Монте-Карло

$$Q^{\pi}_{\text{tar:MC}}{(s, a)}=\frac{1}{N}\sum_{i=1}^{N}R(\tau_{i}).$$

Отметим, что это уравнение применимо к любому набору траекторий с любой начальной парой $(s,a)$.

- Агенту приходится ждать конца эпизода, прежде чем можно будет воспользоваться для обучения какими-нибудь данными из него.
- Для расчета $Q^{\pi}_{\text{tar:MC}}{(s, a)}$ нужны вознаграждения для остальной части траектории, начиная с $(s,a)$. Эпизоды могут состоять из большого количества временных шагов, ограниченного $T$, что задерживает обучение. Этим обучловлено использование TD-обучения для настройки $Q$-значений.

### TD-обучение

$Q$-значения для текущего временного шага могут быть определены через $Q$-значения для следующего временного шага (рекурсивно):
$$
Q^{\pi}(s,a)=\mathbb{E}_{s'\sim p(s'| s, a),\, r\sim \mathcal R (s,a, s')}\left[r+\gamma\mathbb{E}_{a'\sim\pi(s')}\left[Q^{\pi}(s',a')\right]\right].
$$

<img src="pictures/bellman.jpg" alt="Richard E. Bellman" width="300">

Ричард Эрнест Беллман (Richard E. Bellman)

(1920–1984)

Пусть существует нейронная сеть $Q_{\theta}$ представляющая $Q$-функцию. В TD обучении $Q^{\pi}_{\text{tar}}{(s_t,a_t)}$ через оценку правой части уравнения Беллмана с помощью $Q_{\theta}$. На каждой итерации обучения значение $\hat Q^{\pi}_{\text{tar}}{(s_t,a_t)}$ обновляется и $\hat Q^{\pi}_{\text{tar}}{(s_t,a_t)}\rightarrow Q^{\pi}_{\text{tar}}{(s_t,a_t)}$.

### Нюансы

Использование уравнения Беллмана для построения $Q^{\pi}_{\text{tar}}{(s_t,a_t)}$ сопрежено с некоторыми проблемами:
- Мы применяем два мат ожидания:
    - внешнее $\mathbb{E}_{s'\sim p(s' | s, a),\, r\sim \mathcal {R} (s,a,s')} [\,...]$ по следующим состояниям и вознаграждениям.
      Используем только один параметр для оценки распределения по следующим состояниям $s'$ и вознаграждениям $r$, это дает возможность отбросить внешнее мат ожидание, тогда
      $$
      Q^{\pi}(s,a)=r+\gamma\mathbb{E}_{a'\sim\pi(s')} [Q^{\pi}(s',a')].
      $$
    - внутреннее $\mathbb{E}_{a'\sim \pi (s')}[\, ...]$ по действиям.
      $$
      Q^{\pi}(s,a)\approx r+\gamma Q^{\pi}(s',a')=Q^{\pi}_{\text{tar:SARSA}}(s,a).
      $$
      DQN:
      $$
      Q^{\pi}(s,a)\approx r+ \max_{a'_{i}}Q^{\pi}(s',a'_i)=Q^{\pi}_{\text{tar:DQN}}(s,a).
      $$

- Extented SARSA
  - Саттон Р. С., Барто Э. Дж. Обучение с подкреплением: Введение. 2-е изд. М.: ДМК Пресс, 2020. 552 с.

### Пример

Примера пока что нет.

### Выбор действий в SARSA

1. Инициализируем случайным образом нейронную сеть с параметром $\theta$, которая представляет $Q$-функцию и обозначена $Q^{\pi}(s,a;\theta)$.
2. Повторяем следующие шаги до тех пор, пока агент не перестанет улучшаться. Улучшение оценивается как сумма вознаграждений, полученных за эпизод.
    1. Используем $Q^{\pi}(s,a;\theta)$ для действий в среде, действуя жадно по отношению к $Q$-значениям. Сохраняем все прецеденты $(s,a,r,s')$.
    2. Используем сохраненные прецеденты для обновления $Q^{\pi}(s,a;\theta)$ с помощью уравнения Беллмана в методе SARSA. Это улучшается оценку $Q$-функции,
       что в свою очередь, совершенствует стратегию, так как оценки $Q$-значений теперь лучше.

## Исследование и использование

Была доказана сходимость TD-обучения к оптимальной $Q$-функции для линейной аппроксимации функции.

- Sutton R. S. Learning to predict by the methods of temporal differences // Machine Learning. 1988. Vol. 3 (1), P. 9–44. DOI: [10.1007/BF00115009](https://doi.org/10.1007/BF00115009).
- Tsitsiklis J. N. An Analysis of Temporal-Difference Learning with Function Approximation // IEEE Transactions on Automatic Control. 1997. Vol. 42 (5). P. 674–690. DOI: [10.1109/9.580874](https://doi.org/10.1109/9.580874).

Сходимость TD-обучения при переходе к нелинейной аппроксимации функции не гарантируется.

- https://www.youtube.com/watch?v=k1vNh4rNYec
- Mnih V., Kavukcuoglu K., Silver D., Graves A., Antonoglou I., Wierstra D. Riedmiller M. A. Playing Atari with Deep Reinforcement Learning. 2013. https://arxiv.org/abs/1312.5602
- Tesauro G. Temporal Difference Learning and TD-Gammon // Communications of the ACM 38.3
    - https://github.com/AestheticVoyager/Temporal-Difference-Learning

## Алгоритм SARSA

Оценка $Q$-функции $\hat Q^{\pi}(s,a)$ параметризована сетью с параметрами $\theta$, обозначенной как $Q^{\pi_{\theta}}$, так что $\hat Q^{\pi}(s,a)=Q^{\pi_{\theta}}(s,a).$

1. Инициализация скорости обучения $\alpha$.
2. Инициализация $\varepsilon$.
3. Инициализация параметров сети $\theta$ случайными значениями.
4. **for** $m=1,2,\ldots, MAX\_STEPS$ **do**
5. $\quad$ Накопить $N$ прецедентов $(s_i,a_i,r_i,s_i',a_i')$, используя текущую $\varepsilon$-жадную стратегию.
6. $\quad$ **for** $i = 1,2,\ldots, N$ **do**
7. $\quad\quad$ # Расситать целевые $Q$-значения для каждого прецедента
8. $\quad\quad$ $y_i=r_i+\delta_{s_i'}\gamma Q^{\pi_{\theta}}(s_i',a_i')$, где $\delta_{s_i'}=\theta$, если $s_i'$ — конечное состояние, иначе 1
9. $\quad$ **end for**
10. $\quad$ # Расчет функции потерь, например, с помощью скп
11. $\quad$ $L(\theta)=\dfrac{1}{N}\sum_{i}\left(y_i-Q^{\pi_{\theta}}(s_i,a_i)\right)^2$
12. $\quad$ # Обновление параметров сети
13. $\quad$ $\theta=\theta - \alpha \nabla_{\theta} J(\theta)$
14. $\quad$ Уменьшение $\varepsilon$
15. **end for**

## Алгоритм обучения по актуальному опыту

Рассмотрим TD-обновление в SARSA
$$
Q^{\pi_1}(s,a)\approx r + \gamma Q^{\pi_1}\left(s',a_1'\right).
$$

Здесь неуместны прецеденты, сгенерированные с помощью других стратегий.

Пусть существует прецедент $(s,a,r,s',a'_2)$, порожденный стратегией $\pi_2$.

$a'_2$ не обязательно будет таким же, как $a'_1$.
Если это так, то $Q^{\pi_1}(s,a)$ не будет отражать ожидаемое суммарное будущее дисконтированное вознаграждение за выбор действия $a$ в состоянии $s$ в соответствии с $\pi_1$:
$$
Q^{\pi_1}_{\text{tar}}(s,a)=r+\gamma Q^{\pi_1}(s',a'_1)\neq r + \gamma Q^{\pi_1}(s',a'_2).
$$


## Реализация SARSA

Реализация SARSA состоит из трех основных компонентов.

- epsilon_gredy определяет метод для действий в среде согласно текущей стратегии.
- calc_q_loss использует накопленные прецеденты для расчета целевых $Q$-значений и соответствующих значений функции потерь.
  $$
  \hat Q^{\pi}(s,a)=r+\gamma\hat Q^{\pi}(s',a')=Q^{\pi}_{\text{tar}}(s,a).
  $$
- train производит одно обновление параметров сети полезности с помощью $Q$-функции потерь. Происходит обновление всех значимых переменных, таких как explore_var($\varepsilon$).

См. реализацию из SLM-Lab.